In [13]:
import mlflow
import pandas as pd

# Set the tracking URI to the directory where mlruns is located
mlflow.set_tracking_uri("file:///Users/pabloparedes/Documents/prophet_GB_demand_forecasting_workflow/mlruns")

## Handling Artifacts

### From a run

In [10]:
# Getting run saved artifacts from the client.
from mlflow.tracking import MlflowClient

client = MlflowClient()
run_id = "3744bf44ae2746b4a6a7c7660334e462"

# List artifacts
artifacts = client.list_artifacts(run_id)
for artifact in artifacts:
    print(artifact.path)

added_model_outputs
datasets
forecast_outputs
graphs
hyperopt
interim_model_outputs
main_model_outputs
model_non_domestic_monthly


In [14]:
artifact_path = "datasets/input_df.parquet"

# Load the artifact directly
local_path = client.download_artifacts(run_id, artifact_path)
artifact_df = pd.read_parquet(local_path)
artifact_df

,ds,sector,y,gdp,abnormal,covid_1,covid_2,covid_3,is_jan,is_feb,...,is_apr,is_may,is_jun,is_jul,is_aug,is_sep,is_oct,is_nov,is_dec,weekdays
0,2015-01-01,Non-domestic,17535.797012,91.6870,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,22
1,2015-02-01,Non-domestic,16235.838323,91.9610,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,20
2,2015-03-01,Non-domestic,17104.171944,92.0195,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,22
3,2015-04-01,Non-domestic,15249.697840,92.5513,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,22
4,2015-05-01,Non-domestic,15339.201789,92.3066,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,2023-03-01,Non-domestic,15327.936243,102.2569,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,23
99,2023-04-01,Non-domestic,13401.969106,102.4903,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,20
100,2023-05-01,Non-domestic,13328.137841,102.2648,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,23
101,2023-06-01,Non-domestic,13275.691529,103.0142,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,22


### From a Model

## Validating the model before deployment

### Way 1

In [15]:
from mlflow.models import validate_serving_input
from mlflow.models import convert_input_example_to_serving_input

model_uri = 'runs:/3744bf44ae2746b4a6a7c7660334e462/model_non_domestic_monthly'

serving_payload = convert_input_example_to_serving_input(artifact_df)

validate_serving_input(model_uri, serving_payload)

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,abnormal,abnormal_lower,abnormal_upper,covid_1,...,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekdays,weekdays_lower,weekdays_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2015-01-01,16411.191278,17442.594714,17925.282441,16411.191278,16411.191278,0.0,0.0,0.0,0.0,...,0.077263,0.077263,0.077263,0.001074,0.001074,0.001074,0.0,0.0,0.0,17679.172937
1,2015-02-01,16368.562852,15694.092677,16167.910807,16368.562852,16368.562852,0.0,0.0,0.0,0.0,...,-0.026176,-0.026176,-0.026176,-0.006830,-0.006830,-0.006830,0.0,0.0,0.0,15940.094031
2,2015-03-01,16330.059757,16839.668502,17307.473729,16330.059757,16330.059757,0.0,0.0,0.0,0.0,...,0.046239,0.046239,0.046239,0.001074,0.001074,0.001074,0.0,0.0,0.0,17085.150543
3,2015-04-01,16287.431331,15195.672174,15665.853674,16287.431331,16287.431331,0.0,0.0,0.0,0.0,...,-0.052563,-0.052563,-0.052563,0.001074,0.001074,0.001074,0.0,0.0,0.0,15431.317620
4,2015-05-01,16246.178015,15033.939179,15524.399963,16246.178015,16246.178015,0.0,0.0,0.0,0.0,...,-0.059766,-0.059766,-0.059766,-0.002878,-0.002878,-0.002878,0.0,0.0,0.0,15275.204093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,2023-03-01,13060.307813,14751.486007,15222.566412,13060.307813,13060.307813,0.0,0.0,0.0,0.0,...,0.147196,0.147196,0.147196,0.005027,0.005027,0.005027,0.0,0.0,0.0,14982.726727
99,2023-04-01,13022.137192,13232.752156,13687.355062,13022.137192,13022.137192,0.0,0.0,0.0,0.0,...,0.033709,0.033709,0.033709,-0.006830,-0.006830,-0.006830,0.0,0.0,0.0,13461.099846
100,2023-05-01,12985.197881,13300.255859,13779.305177,12985.197881,12985.197881,0.0,0.0,0.0,0.0,...,0.042497,0.042497,0.042497,0.005027,0.005027,0.005027,0.0,0.0,0.0,13537.025741
101,2023-06-01,12947.027259,13029.603075,13506.446729,12947.027259,12947.027259,0.0,0.0,0.0,0.0,...,0.025348,0.025348,0.025348,0.001074,0.001074,0.001074,0.0,0.0,0.0,13275.211977


### Way 2

In [19]:
from mlflow.models import validate_serving_input

model_uri = 'runs:/3744bf44ae2746b4a6a7c7660334e462/model_non_domestic_monthly'

# The model is logged with an input example. MLflow converts it into the serving payload format for the deployed model endpoint, and saves it to 'serving_input_payload.json'
serving_payload = """{
  "dataframe_split": {
    "columns": [
      "ds",
      "sector",
      "y",
      "gdp",
      "abnormal",
      "covid_1",
      "covid_2",
      "covid_3",
      "is_jan",
      "is_feb",
      "is_leap",
      "is_mar",
      "is_apr",
      "is_may",
      "is_jun",
      "is_jul",
      "is_aug",
      "is_sep",
      "is_oct",
      "is_nov",
      "is_dec",
      "weekdays"
    ],
    "data": [
      [
        "2015-01-01T00:00:00",
        "Non-domestic",
        17535.797012021383,
        91.687,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2015-02-01T00:00:00",
        "Non-domestic",
        16235.838323447319,
        91.961,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        20
      ],
      [
        "2015-03-01T00:00:00",
        "Non-domestic",
        17104.1719443741,
        92.0195,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2015-04-01T00:00:00",
        "Non-domestic",
        15249.69783982746,
        92.5513,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2015-05-01T00:00:00",
        "Non-domestic",
        15339.201789410707,
        92.3066,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        21
      ],
      [
        "2015-06-01T00:00:00",
        "Non-domestic",
        15197.12042075396,
        92.2139,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2015-07-01T00:00:00",
        "Non-domestic",
        15615.678911861447,
        92.6526,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        23
      ],
      [
        "2015-08-01T00:00:00",
        "Non-domestic",
        15090.967390151669,
        92.5197,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        21
      ],
      [
        "2015-09-01T00:00:00",
        "Non-domestic",
        15302.345966276074,
        92.6516,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        22
      ],
      [
        "2015-10-01T00:00:00",
        "Non-domestic",
        16235.177576709475,
        93.0295,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        22
      ],
      [
        "2015-11-01T00:00:00",
        "Non-domestic",
        16283.086920936941,
        92.8692,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        21
      ],
      [
        "2015-12-01T00:00:00",
        "Non-domestic",
        15819.140273243129,
        93.2685,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        23
      ],
      [
        "2016-01-01T00:00:00",
        "Non-domestic",
        16945.416038898846,
        93.2419,
        1,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        21
      ],
      [
        "2016-02-01T00:00:00",
        "Non-domestic",
        16241.990563928472,
        93.5345,
        0,
        0,
        0,
        0,
        0,
        1,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        21
      ],
      [
        "2016-03-01T00:00:00",
        "Non-domestic",
        16747.770106517117,
        93.7219,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        23
      ],
      [
        "2016-04-01T00:00:00",
        "Non-domestic",
        15532.868712605696,
        94.2234,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        21
      ],
      [
        "2016-05-01T00:00:00",
        "Non-domestic",
        15099.042846568918,
        94.0166,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2016-06-01T00:00:00",
        "Non-domestic",
        14889.02638737926,
        94.1336,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2016-07-01T00:00:00",
        "Non-domestic",
        15154.134680655361,
        94.2428,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        21
      ],
      [
        "2016-08-01T00:00:00",
        "Non-domestic",
        15017.93188731707,
        94.5378,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        23
      ],
      [
        "2016-09-01T00:00:00",
        "Non-domestic",
        15222.03575570688,
        95.0462,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        22
      ],
      [
        "2016-10-01T00:00:00",
        "Non-domestic",
        15715.778588721996,
        94.737,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        21
      ],
      [
        "2016-11-01T00:00:00",
        "Non-domestic",
        16635.855027275113,
        95.2482,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        22
      ],
      [
        "2016-12-01T00:00:00",
        "Non-domestic",
        16060.753973744511,
        95.9471,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        22
      ],
      [
        "2017-01-01T00:00:00",
        "Non-domestic",
        17291.69094410736,
        96.0813,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2017-02-01T00:00:00",
        "Non-domestic",
        15464.933026626855,
        96.1087,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        20
      ],
      [
        "2017-03-01T00:00:00",
        "Non-domestic",
        16381.100223069192,
        96.2108,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        23
      ],
      [
        "2017-04-01T00:00:00",
        "Non-domestic",
        14564.103629076368,
        96.613,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        20
      ],
      [
        "2017-05-01T00:00:00",
        "Non-domestic",
        15270.481727135519,
        96.7602,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        23
      ],
      [
        "2017-06-01T00:00:00",
        "Non-domestic",
        14902.13716954952,
        96.7748,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2017-07-01T00:00:00",
        "Non-domestic",
        14969.40889914004,
        96.9762,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        21
      ],
      [
        "2017-08-01T00:00:00",
        "Non-domestic",
        14812.090143964177,
        97.1614,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        23
      ],
      [
        "2017-09-01T00:00:00",
        "Non-domestic",
        14811.92423839234,
        97.5765,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        21
      ],
      [
        "2017-10-01T00:00:00",
        "Non-domestic",
        15597.383459360466,
        97.6088,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        22
      ],
      [
        "2017-11-01T00:00:00",
        "Non-domestic",
        16256.153371827832,
        97.8274,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        22
      ],
      [
        "2017-12-01T00:00:00",
        "Non-domestic",
        16120.861639907607,
        98.0528,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        21
      ],
      [
        "2018-01-01T00:00:00",
        "Non-domestic",
        17103.993563156284,
        97.8321,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        23
      ],
      [
        "2018-02-01T00:00:00",
        "Non-domestic",
        15772.397959470072,
        97.8775,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        20
      ],
      [
        "2018-03-01T00:00:00",
        "Non-domestic",
        17171.712511600737,
        98.0007,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2018-04-01T00:00:00",
        "Non-domestic",
        15422.241142188357,
        97.769,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        21
      ],
      [
        "2018-05-01T00:00:00",
        "Non-domestic",
        15212.36589980497,
        98.0835,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        23
      ],
      [
        "2018-06-01T00:00:00",
        "Non-domestic",
        14773.972437382632,
        98.4296,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        21
      ],
      [
        "2018-07-01T00:00:00",
        "Non-domestic",
        15634.647125218264,
        98.2858,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2018-08-01T00:00:00",
        "Non-domestic",
        15219.735326985627,
        98.4504,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        23
      ],
      [
        "2018-09-01T00:00:00",
        "Non-domestic",
        14822.914976657226,
        98.5964,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        20
      ],
      [
        "2018-10-01T00:00:00",
        "Non-domestic",
        16074.946975413706,
        98.495,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        23
      ],
      [
        "2018-11-01T00:00:00",
        "Non-domestic",
        16475.264261675842,
        98.7516,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        22
      ],
      [
        "2018-12-01T00:00:00",
        "Non-domestic",
        15929.549819517762,
        98.4909,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        21
      ],
      [
        "2019-01-01T00:00:00",
        "Non-domestic",
        17367.161905534223,
        98.8571,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        23
      ],
      [
        "2019-02-01T00:00:00",
        "Non-domestic",
        15392.896555366387,
        99.5299,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        20
      ],
      [
        "2019-03-01T00:00:00",
        "Non-domestic",
        16412.596075245885,
        99.5556,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        21
      ],
      [
        "2019-04-01T00:00:00",
        "Non-domestic",
        15031.056261358572,
        99.3054,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2019-05-01T00:00:00",
        "Non-domestic",
        15062.361987705226,
        99.5938,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        23
      ],
      [
        "2019-06-01T00:00:00",
        "Non-domestic",
        14476.3667570368,
        100.0858,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        20
      ],
      [
        "2019-07-01T00:00:00",
        "Non-domestic",
        15215.781615178885,
        100.5729,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        23
      ],
      [
        "2019-08-01T00:00:00",
        "Non-domestic",
        14771.34709255847,
        100.2885,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2019-09-01T00:00:00",
        "Non-domestic",
        14545.56104452236,
        100.6168,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        21
      ],
      [
        "2019-10-01T00:00:00",
        "Non-domestic",
        15944.742074249458,
        100.4694,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        23
      ],
      [
        "2019-11-01T00:00:00",
        "Non-domestic",
        16308.201761302935,
        100.3002,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        21
      ],
      [
        "2019-12-01T00:00:00",
        "Non-domestic",
        15883.801367521526,
        100.8246,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        22
      ],
      [
        "2020-01-01T00:00:00",
        "Non-domestic",
        16674.986266349246,
        100.8145,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        23
      ],
      [
        "2020-02-01T00:00:00",
        "Non-domestic",
        15733.442158964705,
        100.444,
        0,
        0,
        0,
        0,
        0,
        1,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        20
      ],
      [
        "2020-03-01T00:00:00",
        "Non-domestic",
        15138.90317703463,
        92.7769,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2020-04-01T00:00:00",
        "Non-domestic",
        11078.320199328047,
        74.5691,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2020-05-01T00:00:00",
        "Non-domestic",
        11384.798787873327,
        76.1555,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        21
      ],
      [
        "2020-06-01T00:00:00",
        "Non-domestic",
        11963.144241665106,
        83.5776,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2020-07-01T00:00:00",
        "Non-domestic",
        13036.934726728512,
        89.126,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        23
      ],
      [
        "2020-08-01T00:00:00",
        "Non-domestic",
        13455.367422179861,
        91.8198,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        21
      ],
      [
        "2020-09-01T00:00:00",
        "Non-domestic",
        13638.752415087776,
        93.026,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        22
      ],
      [
        "2020-10-01T00:00:00",
        "Non-domestic",
        14697.404390294845,
        93.5677,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        22
      ],
      [
        "2020-11-01T00:00:00",
        "Non-domestic",
        14202.190867820562,
        91.6681,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        21
      ],
      [
        "2020-12-01T00:00:00",
        "Non-domestic",
        14620.593398355504,
        92.8976,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        23
      ],
      [
        "2021-01-01T00:00:00",
        "Non-domestic",
        15022.401623092968,
        90.3167,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        21
      ],
      [
        "2021-02-01T00:00:00",
        "Non-domestic",
        13577.589939312118,
        91.2726,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        20
      ],
      [
        "2021-03-01T00:00:00",
        "Non-domestic",
        14531.296655293414,
        93.3356,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        23
      ],
      [
        "2021-04-01T00:00:00",
        "Non-domestic",
        13422.333138252114,
        96.7661,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2021-05-01T00:00:00",
        "Non-domestic",
        13799.357281940504,
        98.2848,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        21
      ],
      [
        "2021-06-01T00:00:00",
        "Non-domestic",
        13502.378548654833,
        99.4212,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2021-07-01T00:00:00",
        "Non-domestic",
        13947.917457683272,
        98.8991,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2021-08-01T00:00:00",
        "Non-domestic",
        13680.1253207318,
        99.7767,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2021-09-01T00:00:00",
        "Non-domestic",
        14021.640528254951,
        100.3606,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        22
      ],
      [
        "2021-10-01T00:00:00",
        "Non-domestic",
        14791.953418054753,
        100.7081,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        21
      ],
      [
        "2021-11-01T00:00:00",
        "Non-domestic",
        15362.751975689156,
        101.4433,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        22
      ],
      [
        "2021-12-01T00:00:00",
        "Non-domestic",
        15291.688858270194,
        100.9219,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        23
      ],
      [
        "2022-01-01T00:00:00",
        "Non-domestic",
        15941.34095203516,
        101.4331,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        21
      ],
      [
        "2022-02-01T00:00:00",
        "Non-domestic",
        14451.6372918677,
        101.966,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        20
      ],
      [
        "2022-03-01T00:00:00",
        "Non-domestic",
        15388.597764924643,
        102.1056,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        23
      ],
      [
        "2022-04-01T00:00:00",
        "Non-domestic",
        13809.78202913026,
        101.8883,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        21
      ],
      [
        "2022-05-01T00:00:00",
        "Non-domestic",
        13847.973745409136,
        102.5782,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2022-06-01T00:00:00",
        "Non-domestic",
        13375.251451208456,
        101.5609,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2022-07-01T00:00:00",
        "Non-domestic",
        14027.473769230404,
        101.9985,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        21
      ],
      [
        "2022-08-01T00:00:00",
        "Non-domestic",
        14013.537818078,
        102.0639,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        23
      ],
      [
        "2022-09-01T00:00:00",
        "Non-domestic",
        13476.443833844725,
        101.4602,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        22
      ],
      [
        "2022-10-01T00:00:00",
        "Non-domestic",
        14312.351866932911,
        102.1466,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        21
      ],
      [
        "2022-11-01T00:00:00",
        "Non-domestic",
        14721.70672668738,
        102.205,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        22
      ],
      [
        "2022-12-01T00:00:00",
        "Non-domestic",
        15132.771738277614,
        101.8905,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        22
      ],
      [
        "2023-01-01T00:00:00",
        "Non-domestic",
        15602.615927347004,
        102.4213,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2023-02-01T00:00:00",
        "Non-domestic",
        13850.088439780347,
        102.5735,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        20
      ],
      [
        "2023-03-01T00:00:00",
        "Non-domestic",
        15327.936242667789,
        102.2569,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        23
      ],
      [
        "2023-04-01T00:00:00",
        "Non-domestic",
        13401.96910588743,
        102.4903,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        20
      ],
      [
        "2023-05-01T00:00:00",
        "Non-domestic",
        13328.137840592095,
        102.2648,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        23
      ],
      [
        "2023-06-01T00:00:00",
        "Non-domestic",
        13275.691529254138,
        103.0142,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        0,
        22
      ],
      [
        "2023-07-01T00:00:00",
        "Non-domestic",
        13341.053288400819,
        102.3513,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        1,
        0,
        0,
        0,
        0,
        0,
        21
      ]
    ]
  }
}"""

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,abnormal,abnormal_lower,abnormal_upper,covid_1,...,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekdays,weekdays_lower,weekdays_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2015-01-01,16411.191278,17445.138543,17911.995528,16411.191278,16411.191278,0.0,0.0,0.0,0.0,...,0.077263,0.077263,0.077263,0.001074,0.001074,0.001074,0.0,0.0,0.0,17679.172937
1,2015-02-01,16368.562852,15708.507183,16170.940411,16368.562852,16368.562852,0.0,0.0,0.0,0.0,...,-0.026176,-0.026176,-0.026176,-0.006830,-0.006830,-0.006830,0.0,0.0,0.0,15940.094031
2,2015-03-01,16330.059757,16858.891419,17319.129207,16330.059757,16330.059757,0.0,0.0,0.0,0.0,...,0.046239,0.046239,0.046239,0.001074,0.001074,0.001074,0.0,0.0,0.0,17085.150543
3,2015-04-01,16287.431331,15198.945136,15673.328176,16287.431331,16287.431331,0.0,0.0,0.0,0.0,...,-0.052563,-0.052563,-0.052563,0.001074,0.001074,0.001074,0.0,0.0,0.0,15431.317620
4,2015-05-01,16246.178015,15026.939636,15513.352953,16246.178015,16246.178015,0.0,0.0,0.0,0.0,...,-0.059766,-0.059766,-0.059766,-0.002878,-0.002878,-0.002878,0.0,0.0,0.0,15275.204093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,2023-03-01,13060.307813,14735.078581,15209.966422,13060.307813,13060.307813,0.0,0.0,0.0,0.0,...,0.147196,0.147196,0.147196,0.005027,0.005027,0.005027,0.0,0.0,0.0,14982.726727
99,2023-04-01,13022.137192,13222.921975,13710.828375,13022.137192,13022.137192,0.0,0.0,0.0,0.0,...,0.033709,0.033709,0.033709,-0.006830,-0.006830,-0.006830,0.0,0.0,0.0,13461.099846
100,2023-05-01,12985.197881,13301.004430,13770.619997,12985.197881,12985.197881,0.0,0.0,0.0,0.0,...,0.042497,0.042497,0.042497,0.005027,0.005027,0.005027,0.0,0.0,0.0,13537.025741
101,2023-06-01,12947.027259,13045.318973,13531.003400,12947.027259,12947.027259,0.0,0.0,0.0,0.0,...,0.025348,0.025348,0.025348,0.001074,0.001074,0.001074,0.0,0.0,0.0,13275.211977


## Ways of calling the Model / Run

In [110]:
mlflow.pyfunc.load_model('runs:/3744bf44ae2746b4a6a7c7660334e462/model_non_domestic_monthly')

mlflow.pyfunc.loaded_model:
  artifact_path: model_non_domestic_monthly
  flavor: mlflow.prophet
  run_id: 3744bf44ae2746b4a6a7c7660334e462

In [111]:
mlflow.pyfunc.load_model("models:/gb_non_dom_monthly/1")

mlflow.pyfunc.loaded_model:
  artifact_path: model_non_domestic_monthly
  flavor: mlflow.prophet
  run_id: cce09a40fdae437f9ade146c39c692a1

In [112]:
mlflow.prophet.load_model("models:/gb_non_dom_monthly/1")

## Using the model: Paramteres, Predictions

In [81]:
logged_model = 'runs:/3744bf44ae2746b4a6a7c7660334e462/model_non_domestic_monthly'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

print(loaded_model)
print(loaded_model.get_raw_model())
print("From Metadata:")
print("input schema: ",loaded_model.metadata.get_input_schema())
print("output schema: ", loaded_model.metadata.get_output_schema())
print("artifact path: ", loaded_model.metadata.artifact_path)
print("flavors (model, python, versions): ", loaded_model.metadata.flavors)
print("model identifier uuid: ", loaded_model.metadata.model_uuid)
print("model's run id: ", loaded_model.metadata.run_id)

mlflow.pyfunc.loaded_model:
  artifact_path: model_non_domestic_monthly
  flavor: mlflow.prophet
  run_id: 3744bf44ae2746b4a6a7c7660334e462

From Metadata:
input schema:  ['ds': datetime (required), 'sector': string (required), 'y': double (required), 'gdp': double (required), 'abnormal': long (required), 'covid_1': long (required), 'covid_2': long (required), 'covid_3': long (required), 'is_jan': long (required), 'is_feb': long (required), 'is_leap': long (required), 'is_mar': long (required), 'is_apr': long (required), 'is_may': long (required), 'is_jun': long (required), 'is_jul': long (required), 'is_aug': long (required), 'is_sep': long (required), 'is_oct': long (required), 'is_nov': long (required), 'is_dec': long (required), 'weekdays': long (required)]
output schema:  ['ds': datetime (required), 'y': double (optional), 'sector': string (required), 'nominal_gdp': double (required), 'nominal_weekdays': long (required), 'trend': double (required), 'yhat': double (required), 'yhat

In [46]:
# Predict on a Pandas DataFrame.
loaded_model.predict(artifact_df)

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,abnormal,abnormal_lower,abnormal_upper,covid_1,...,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekdays,weekdays_lower,weekdays_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2015-01-01,16411.191278,17440.812217,17928.852872,16411.191278,16411.191278,0.0,0.0,0.0,0.0,...,0.077263,0.077263,0.077263,0.001074,0.001074,0.001074,0.0,0.0,0.0,17679.172937
1,2015-02-01,16368.562852,15681.070942,16180.594697,16368.562852,16368.562852,0.0,0.0,0.0,0.0,...,-0.026176,-0.026176,-0.026176,-0.006830,-0.006830,-0.006830,0.0,0.0,0.0,15940.094031
2,2015-03-01,16330.059757,16830.354031,17326.083429,16330.059757,16330.059757,0.0,0.0,0.0,0.0,...,0.046239,0.046239,0.046239,0.001074,0.001074,0.001074,0.0,0.0,0.0,17085.150543
3,2015-04-01,16287.431331,15184.687615,15680.394658,16287.431331,16287.431331,0.0,0.0,0.0,0.0,...,-0.052563,-0.052563,-0.052563,0.001074,0.001074,0.001074,0.0,0.0,0.0,15431.317620
4,2015-05-01,16246.178015,15027.245789,15516.169293,16246.178015,16246.178015,0.0,0.0,0.0,0.0,...,-0.059766,-0.059766,-0.059766,-0.002878,-0.002878,-0.002878,0.0,0.0,0.0,15275.204093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,2023-03-01,13060.307813,14756.770427,15224.609177,13060.307813,13060.307813,0.0,0.0,0.0,0.0,...,0.147196,0.147196,0.147196,0.005027,0.005027,0.005027,0.0,0.0,0.0,14982.726727
99,2023-04-01,13022.137192,13222.360792,13705.149895,13022.137192,13022.137192,0.0,0.0,0.0,0.0,...,0.033709,0.033709,0.033709,-0.006830,-0.006830,-0.006830,0.0,0.0,0.0,13461.099846
100,2023-05-01,12985.197881,13294.678140,13764.277921,12985.197881,12985.197881,0.0,0.0,0.0,0.0,...,0.042497,0.042497,0.042497,0.005027,0.005027,0.005027,0.0,0.0,0.0,13537.025741
101,2023-06-01,12947.027259,13035.135520,13505.261881,12947.027259,12947.027259,0.0,0.0,0.0,0.0,...,0.025348,0.025348,0.025348,0.001074,0.001074,0.001074,0.0,0.0,0.0,13275.211977


## Finding model through Model Name

In [90]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
registered_model_name = "gb_non_dom_monthly"

for i in client.get_registered_model(registered_model_name):
  print(i)

('aliases', {'newcomer': '1'})
('creation_timestamp', 1728586443862)
('description', '(Dummy Description)\nLatest GB Non-Domestic Monthly Forecasting Model')
('last_updated_timestamp', 1728661165611)
('latest_versions', [<ModelVersion: aliases=[], creation_timestamp=1728586943491, current_stage='None', description='', last_updated_timestamp=1728586943491, name='gb_non_dom_monthly', run_id='cce09a40fdae437f9ade146c39c692a1', run_link='', source='models:/gb_non_dom_monthly/1', status='READY', status_message=None, tags={'user': 'pablo'}, user_id=None, version=2>])
('name', 'gb_non_dom_monthly')
('tags', {'env': 'prod'})
